In [1]:
import explogreg
import torch
from pyro.infer import MCMC, NUTS
import pyro.optim as optim

%load_ext autoreload
%autoreload 2

/Users/elenab/Library/r-miniconda-arm64/envs/lineagt-env/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
torch.manual_seed(0)
L = 3
T = 5 # 6  # Number of time points
# x = torch.linspace(0, 10, T).reshape((T,1))  # Time variable
x = torch.tensor([0, 56, 84, 112, 168, 280]).unsqueeze(1)  # Time variable

# y = torch.tensor([[0, 138, 188, 565, 572, 539], [0, 238, 548, 793, 649, 629], [0, 28, 128, 283, 405, 486]]).T  # Simulated binary outcomes (logistic regression)
y = torch.tensor([[0, 3.5, 6, 8, 1, 271], 
                  [0, 20, 11, 7, 272, 317], 
                  [0, 2, 2, 5, 2, 245]]).T  # Simulated binary outcomes (logistic regression)

In [3]:
print(x)
print(y)

tensor([[  0],
        [ 56],
        [ 84],
        [112],
        [168],
        [280]])
tensor([[  0.0000,   0.0000,   0.0000],
        [  3.5000,  20.0000,   2.0000],
        [  6.0000,  11.0000,   2.0000],
        [  8.0000,   7.0000,   5.0000],
        [  1.0000, 272.0000,   2.0000],
        [271.0000, 317.0000, 245.0000]])


In [ ]:
obj = explogreg.Regression(x, y)
# mcmc_exp = obj.train_mcmc(regr="exp")
mcmc_log = obj.train_mcmc(regr="log") #, num_samples=100, warmup_steps=10)

# Get posterior samples
# posterior_samples_exp = mcmc_exp.get_samples()
posterior_samples_log = mcmc_log.get_samples()

Warmup [1]:   0%|          | 0/1500 [00:00, ?it/s]

Warmup [1]:   4%|▍         | 64/1500 [00:01, 77.83it/s, step size=9.38e-04, acc. prob=0.763]

Warmup [1]:   7%|▋         | 110/1500 [00:14,  1.87it/s, step size=1.99e-10, acc. prob=0.718]

Warmup [1]:  11%|█         | 165/1500 [01:28,  1.44s/it, step size=2.92e-05, acc. prob=0.718]

Warmup [1]:  12%|█▏        | 184/1500 [02:01,  1.69s/it, step size=9.57e-05, acc. prob=0.726]

In [ ]:
# mcmc_exp.summary()
mcmc_log.summary()

# Exponential

In [6]:
# import seaborn as sns
# for k, v in posterior_samples_exp.items():
#     print(k)
#     sns.displot(v.numpy())

In [7]:
import matplotlib.pyplot as plt

# Generate new x values (e.g., test points for prediction)
x_test = torch.linspace(0, 10, T)

args = posterior_samples_exp["fitness"][:, None] * x_test.unsqueeze(1)
y_preds = torch.exp(args)

# Compute predictions for each posterior sample
y_mean = y_preds.mean(dim=0)  # Mean prediction
y_lower = torch.quantile(y_preds, 0.025, dim=0)  # 2.5% quantile (
y_upper = torch.quantile(y_preds, 0.975, dim=0)  # 97.5% quantile (upper bound)

# Plot results
plt.figure(figsize=(8, 5))
plt.scatter(x.squeeze(), y[:,0], color="black", label="Observed Data")  # Scatter plot of actual data
plt.plot(x_test, y_mean[:,0], label="Mean Prediction", color="blue")
plt.fill_between(x_test, y_lower[:,0], y_upper[:,0], color="blue", alpha=0.3, label="95% Credible Interval")
plt.xlabel("Time")
plt.ylabel("Probability")
plt.legend()
plt.title("Bayesian Logistic Regression with 95% Credible Intervals")
plt.show()

# Logistic

In [ ]:
import seaborn as sns
for k, v in posterior_samples_log.items():
    try:
        ax = sns.displot(v.numpy())
        ax.set(title=k)
    except:
        pass

In [ ]:
import matplotlib.pyplot as plt

# Generate new x values (e.g., test points for prediction)
x_test = torch.linspace(0, 300, 100)

args = - posterior_samples_log["fitness"][:, None] * x_test.unsqueeze(1)
y_preds = posterior_samples_log["carr_capac"][:, None] / ( 1 + (posterior_samples_log["carr_capac"][:, None] - 1) * torch.exp(args) )

# Compute predictions for each posterior sample
y_mean = y_preds.mean(dim=0)  # Mean prediction
y_lower = torch.quantile(y_preds, 0.05, dim=0)  # 2.5% quantile (
y_upper = torch.quantile(y_preds, 0.95, dim=0)  # 97.5% quantile (upper bound)

# Plot results

for i in range(L):
    plt.figure(figsize=(8, 5))
    plt.scatter(x.squeeze(), y[:,i], color="black", label="Observed Data")  # Scatter plot of actual data
    plt.plot(x_test, y_mean[:,i], label="Mean Prediction", color="blue")
    plt.fill_between(x_test, y_lower[:,i], y_upper[:,i], color="blue", alpha=0.3, label="95% Credible Interval")
    plt.xlabel("Time")
    plt.ylabel("Probability")
    plt.legend()
    plt.title("Bayesian Logistic Regression with 95% Credible Intervals")
    plt.show()